### 函数参数

一个*参数只能出现在函数定义中最后一个位置参数后面

而\**参数只能出现在最后一个参数

有一点要注意的是，在\*参数后面仍然可以定义其他参数：强制关键字参数

In [ ]:
# 任意数量的位置参数和关键字参数
def anyargs(*args, **kwargs):
    print(args) # A tuple
    print(kwargs) # A dict

# 强制关键字参数
def recv(maxsize, *, block):
    pass

### 函数参数注解

In [1]:
def add(x:int, y:int) -> int:
    return x + y

help(add)
print(add.__annotations__)

Help on function add in module __main__:

add(x: int, y: int) -> int

{'x': <class 'int'>, 'y': <class 'int'>, 'return': <class 'int'>}


### 默认参数

默认参数的值仅仅在函数定义的时候赋值一次

In [2]:
_no_value = object()

def spam(a, b=_no_value):
    if b is _no_value:
        print('No b value supplied')
    else:
        print(a, b)
spam(1)
spam(1, 2)
spam(1, None)

No b value supplied
1 2
1 None


In [ ]:
# 避免如下，因为定义时 [] 会保留成全局参数
def spam(a, b=[]): # NO!
    pass

### 匿名函数

有默认值时在定义时捕获，否则在运行时捕获

In [ ]:
add = lambda x, y: x + y
add(2, 3)

In [ ]:
x = 10
a = lambda y: x + y # 运行时捕获
a(10) # x = 10
x = 20
a(10) # x = 20

In [ ]:
x = 10
a = lambda y, x=x: x + y # 定义时捕获
a(10) # x = 10
x = 20
a(10) # x = 10

In [ ]:
# 如果不指定n，n会在运行时捕获
# 4个lambda，n始终为4
funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0))

In [ ]:
# 修改，加上默认值
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

### 补充默认参数

partial

In [ ]:
from functools import partial

def spam(a, b, c, d):
    print(a, b, c, d)

s1 = partial(spam, 1) # a = 1
s2 = partial(spam, d=42) # d = 42
s3 = partial(spam, 1, 2, d=42) # a = 1, b = 2, d = 42

### 闭包

为了简化代码，将除 __init__() 方法外只定义了一个方法的类，转换成一个函数

In [ ]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template

    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))

def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

In [ ]:
# 闭包函数访问外部值
# 使用 nonlocal 访问外部变量
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)

    # Invoke the callback with the result
    callback(result)

def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

handler = make_handler()
apply_async(add, (2, 3), callback=handler)
apply_async(add, ('hello', 'world'), callback=handler)

In [ ]:
# 模拟类
def sample():
    n = 0
    # Closure function
    def func():
        print('n =', n)

    # Accessor methods for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

f = sample()
f()
f.set_n(10)
f()
f.get_n()

In [ ]:
# 模拟类的实例
import sys
class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            # 查询到上层闭包函数
            locals = sys._getframe(1).f_locals

        # Update instance dictionary with callables
        self.__dict__.update((key, value) for key, value in locals.items()
                            if callable(value))

    # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()

# Example use
def Stack():
    items = []
    def push(item):
        items.append(item)

    def pop():
        return items.pop()

    def __len__():
        return len(items)

    return ClosureInstance()

s = Stack()
print(s)
s.push(10)
s.push(20)
s.push('Hello')
print(len(s))
print(s.pop())
print(s.pop())
print(s.pop())

### 改变对象的字符串显示

!s 指明输出使用默认的 __str__() 

!r 格式化代码指明输出使用 __repr__() 来代替默认的 __str__() 

In [ ]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)

    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

p = Pair(3, 4)
print(p) # call __str__
p # call __repr__

In [ ]:
p = Pair(3, 4)
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))

In [ ]:
# 改变format行为
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)
    
d = Date(2012, 12, 21)
print(format(d))
print(format(d, 'mdy'))
print('The date is {:ymd}'.format(d))
print('The date is {:mdy}'.format(d))

### 上下文

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [ ]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    print(resp)
    # conn.__exit__() executes: connection closed

In [ ]:
# 允许多层上下文
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.connections = []

    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()
        
conn = LazyConnection(('www.python.org', 80))
with conn as s1:
    pass
    with conn as s2:
        pass
        # s1 and s2 are independent sockets

### 定义上下文管理器的简单方法

在函数 timethis() 中，yield 之前的代码会在上下文管理器中作为 __enter__() 方法执行， 所有在 yield 之后的代码会作为 __exit__() 方法执行。 如果出现了异常，异常会在yield语句那里抛出。

In [ ]:
import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print('{}: {}'.format(label, end - start))

# Example use
with timethis('counting'):
    n = 10000000
    while n > 0:
        n -= 1

In [ ]:
# 列表对象上的某种事务
@contextmanager
def list_transaction(orig_list):
    working = list(orig_list)
    yield working
    orig_list[:] = working

### \_\_slots\_\_

类似列式存储

当你定义 __slots__ 后，Python就会为实例使用一种更加紧凑的内部表示。 

实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典。

在 __slots__ 中列出的属性名在内部被映射到这个数组的指定小标上。 

使用slots一个不好的地方就是我们不能再给实例添加新的属性了，只能使用在 __slots__ 中定义的那些属性名。

In [ ]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

### 私有属性

以单下划线\_开头的名字，约定是内部实现

使用两个下划线\_\_开头的命名，会导致访问名称变成其他形式，**且无法被继承**

In [3]:
class B:
    def __init__(self):
        self.__private = 0

    def __private_method(self):
        print("_B__private_method")

    def public_method(self):
        self.__private_method()

b = B()
print(b._B__private)
b._B__private_method()
b.public_method()

0
_B__private_method
_B__private_method


In [4]:
# 以双下划线开头的方法无法被重写
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B.__private

    # Does not override B.__private_method()
    def __private_method(self):
        print("_C__private_method")

c = C()
print(c._B__private)
print(c._C__private)
c._B__private_method()
c._C__private_method()

0
1
_B__private_method
_C__private_method


### 父类

方法解析顺序(MRO)列表，是一个简单的所有基类的线性顺序表，查找super时按照MRO顺序来查找

super() = super(Class, self)

当我们调用 super() 的时候，实际上是实例化了一个 super 类，super 包含了两个非常重要的信息: 一个 MRO 以及 MRO 中的一个类，有两种调用方式：

1. super(a_type, obj): 从type(obj)的MRO中，找到a_type，并且查找后一个的类的**实例**。

2. super(type1, type2): 从type2的MRO中，找到type1，并且查找后一个的类，**不绑定实例**。

第二个参数一般是子类，提供了更长的MRO，第一个参数一般是父类（或者祖先类），提供了更短的MRO，从而缩短查找范围。

例如：[A, B, C, D, E, object]中调用super(C, A).foo()，super 只会从 C 之后查找，即: 只会在 D 或 E 或 object 中查找 foo 方法。

In [5]:
# C中的 super 调用 A
# A调用 B，B调用 Base
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A, B):
    def __init__(self):
        super().__init__()  # Only one call to super() here
        print('C.__init__')

c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


In [6]:
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

In [7]:
# A中的 super 实际调用的是 B

class A:
    def spam(self):
        print('A.spam')
        super().spam()

class B:
    def spam(self):
        print('B.spam')

class C(A, B):
    pass

c = C()
c.spam()
C.__mro__

A.spam
B.spam


(__main__.C, __main__.A, __main__.B, object)

### 接口或者抽象基类

@abstractmethod 还能注解静态方法、类方法和 properties，只需保证这个注解紧靠在函数定义前即可

In [ ]:
from abc import ABCMeta, abstractmethod
import io 

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass

    @abstractmethod
    def write(self, data):
        pass
# 继承
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass

    def write(self, data):
        pass

# 注册
# Register the built-in I/O classes as supporting our interface
IStream.register(io.IOBase)

### collections 抽象基类

In [ ]:
import collections

collections.Iterable
collections.Sequence
collections.Container
collections.Sized
collections.Mapping

### 代理访问

代理是一种编程模式，它将某个操作转移给另外一个对象来实现。

注意`__getattr__`对其他双下划线方法不起作用，理由是前面提到的私有方法重命名问题

In [ ]:
# A proxy class that wraps around another object, but
# exposes its public attributes
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        print('getattr:', name)
        return getattr(self._obj, name)

    # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            print('setattr:', name, value)
            setattr(self._obj, name, value)

    # Delegate attribute deletion
    def __delattr__(self, name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:', name)
            delattr(self._obj, name)

In [ ]:
class Spam:
    def __init__(self, x):
        self.x = x

    def bar(self, y):
        print('Spam.bar:', self.x, y)

# Create an instance
s = Spam(2)
# Create a proxy around it
p = Proxy(s)
# Access the proxy
print(p.x)  # Outputs 2
p.bar(3)  # Outputs "Spam.bar: 2 3"
p.x = 37  # Changes s.x to 37

### 多个实例构造器

In [ ]:
import time

class Date:
    """方法一：使用类方法"""
    # Primary constructor
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    # Alternate constructor
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year, t.tm_mon, t.tm_mday)
    
a = Date(2012, 12, 21) # Primary
b = Date.today() # Alternate

### 手动init

In [ ]:
from time import localtime

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        d = cls.__new__(cls)
        t = localtime()
        d.year = t.tm_year
        d.month = t.tm_mon
        d.day = t.tm_mday
        return d

b = Date.today() # Alternate

### 混入类 Mixin

主要利用了super()沿着`__mro__`查找的行为

混入类设计用来掩盖掉其他类中的方法，所以一般不单独做实例化使用

In [8]:
class LoggedMappingMixin:
    """
    Add logging to get/set/delete operations for debugging.
    """
    __slots__ = ()  # 混入类都没有实例变量，因为直接实例化混入类没有任何意义

    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)

    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return super().__setitem__(key, value)

    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)
    
class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['x'] = 23
print(d['x'])
del d['x']
print(LoggedDict.__mro__)

Setting x = 23
Getting x
23
Deleting x
(<class '__main__.LoggedDict'>, <class '__main__.LoggedMappingMixin'>, <class 'dict'>, <class 'object'>)


### 访问者模式

递归实现
https://python3-cookbook.readthedocs.io/zh_CN/latest/c08/p21_implementing_visitor_pattern.html

非递归实现
https://python3-cookbook.readthedocs.io/zh_CN/latest/c08/p22_implementing_visitor_pattern_without_recursion.html

### 弱引用 weakref

循环引用数据结构

一个简单的循环引用数据结构例子就是一个树形结构，双亲节点有指针指向孩子节点，孩子节点又返回来指向双亲节点。

In [ ]:
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    # property that manages the parent as a weak-reference
    @property
    def parent(self):
        # 注意这里 self._parent 后面有括号
        return None if self._parent is None else self._parent()

    @parent.setter
    def parent(self, node):
        # 指向双亲节点使用weakref
        self._parent = weakref.ref(node)
#         self._parent = node

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

root = Node('parent')
c1 = Node('child')
root.add_child(c1)
print(c1.parent)
del root
print(c1.parent) # 

### 比较操作

装饰器 functools.total_ordering 可以用来简化比较操作。 使用它来装饰一个类，你只需定义一个 __eq__() 方法， 外加其他方法(__lt__, __le__, __gt__, or __ge__)中的一个即可。 然后装饰器会自动为你填充其它比较方法。

In [ ]:
from functools import total_ordering

class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width

@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()

    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)

    def add_room(self, room):
        self.rooms.append(room)

    def __str__(self):
        return '{}: {} square foot {}'.format(self.name,
                self.living_space_footage,
                self.style)

    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage

    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage

### 局部变量域

exec中执行的代码，需要从locals中取回

In [ ]:
def test():
    a = 13
    loc = locals()
    exec('b = a + 1')
    b = loc['b']
    print(b)

test()

### 参数签名

对任何涉及到操作函数调用签名的问题，应该使用 inspect 模块中两个类：Signature 和 Parameter

一个签名对象，可以使用它的 bind() 方法很容易的将它绑定到 `*args` 和 `**kwargs` 上去

In [ ]:
from inspect import Signature, Parameter
# Make a signature for a func(x, y=42, *, z=None)
parms = [Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
        Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
        Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]
sig = Signature(parms)
print(sig)

In [ ]:
def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name,value)

# Try various examples
func(1, 2, z=3)

In [ ]:
func(1, 2, 3, 4) # 失败

In [ ]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
            for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

# Example use
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')

class Point(Structure):
    __signature__ = make_sig('x', 'y')

### 框架魔法

sys._getframe() 来获取调用者的模块名

In [225]:
sys._getframe(1).f_globals['__name__']

'IPython.core.interactiveshell'